In [15]:
import pandas as pd
import numpy as np
import os

In [16]:
def cossim(Q, K):
    K_norm = np.linalg.norm(K, axis=1, keepdims=True)
    Q_norm = np.linalg.norm(Q, axis=1, keepdims=True)
    QK_norm = Q_norm @ K_norm.T
    QK = Q @ K.T

    return QK / QK_norm

In [17]:
def processWSI(WSIName):
    keys = np.load("./CM16/keys/combined/train_non_redundant.npy")
    print("Processing: ", WSIName)
    
    lst = [[-1, "max"], [-5, "mean_5"], [-10, "mean_10"], [-15, "mean_15"], [-20, "mean_20"], [-30, "mean_30"], [-50, "mean_50"], [-100, "mean_100"], [-150, "mean_150"], [-200, "mean_200"], [-250, "mean_250"]]
    
    indexes = [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
    
    q = np.load("{}".format(WSIName))
    res = cossim(q, keys)
    for ind, pair in enumerate(lst):
        indexesDesc = np.argsort(np.mean(np.sort(res, axis = 1)[:,pair[0]:], axis = 1))
        np.save("./CM16/ssim_cos_sorted_indexes/compareEachFE/{}_{}".format(pair[1], WSIName.split("/")[-1]), indexesDesc)    

In [ ]:
from multiprocessing import Pool

filePaths = []

for st in ["train", "test"]:
    for className in ["normal", "tumor"]:
        baseDir = "./CM16/CTranspath/cam16CTP/{}/{}/".format(st, className)
        for file in os.listdir(baseDir):
            filePaths.append(baseDir + file)

if __name__ == '__main__':
    with Pool(28) as p:
        print(p.map(processWSI, filePaths))

In [19]:
lst = [[-5, "mean_5"], [-1, "max"], [-10, "mean_10"], [-15, "mean_15"], [-20, "mean_20"], [-30, "mean_30"], [-50, "mean_50"], [-100, "mean_100"], [-150, "mean_150"], [-200, "mean_200"], [-250, "mean_250"]]

repPath = "./CM16/ssim_cos_sorted_indexes/compareEachFE/"
outPath = "./CM16/ssim_cos_sorted_indexes/CTranspath/CM16/"

for l in lst:
    obj = {}
    files = [file for file in os.listdir(repPath) if file.startswith(l[1]+"_")]
    
    for f in files:
        obj["{}".format(f.split(l[1]+"_")[1])] = np.load("{}/{}".format(repPath, f), allow_pickle = True)
    np.save(outPath + "cam16_indexes_dict_CompareEachFE_non_redundant_threshold_0.90_0.80_{}.npy".format(l[1]), obj, allow_pickle = True)